In [1]:
import os
import sys
import torch as t
from torch import Tensor
import einops as e
from ipywidgets import interact
import plotly.express as px
from ipywidgets import interact
from pathlib import Path
from IPython.display import display
from jaxtyping import Float, Int, Bool, Shaped, jaxtyped
import typeguard

# Make sure exercises are in the path
chapter = r"chapter0_fundamentals"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part1_ray_tracing"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from plotly_utils import imshow
from part1_ray_tracing.utils import render_lines_with_plotly, setup_widget_fig_ray, setup_widget_fig_triangle
import part1_ray_tracing.tests as tests

MAIN = __name__ == "__main__"

In [2]:
def make_rays_1d(num_pixels: int, y_limit: float) -> t.Tensor:
    '''
    num_pixels: The number of pixels in the y dimension. Since there is one ray per pixel, this is also the number of rays.
    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both endpoints.

    Returns: shape (num_pixels, num_points=2, num_dim=3) where the num_points dimension contains (origin, direction) and the num_dim dimension contains xyz.

    Example of make_rays_1d(9, 1.0): [
        [[0, 0, 0], [1, -1.0, 0]],
        [[0, 0, 0], [1, -0.75, 0]],
        [[0, 0, 0], [1, -0.5, 0]],
        ...
        [[0, 0, 0], [1, 0.75, 0]],
        [[0, 0, 0], [1, 1, 0]],
    ]
    '''
    a = t.zeros(num_pixels, 1, 3)
    y_values = t.FloatTensor([-y_limit+(x/(num_pixels-1))*2*y_limit for x in range(num_pixels)])
    b = t.cat((t.ones(num_pixels, 1, 1), e.repeat(y_values, "a -> a 1 1"), t.zeros(num_pixels, 1, 1)), -1)
    return t.cat((a, b), 1)
    

rays1d = make_rays_1d(9, 10.0)

fig = render_lines_with_plotly(rays1d)

In [3]:
fig = setup_widget_fig_ray()
display(fig)

@interact
def response(seed=(0, 10, 1), v=(-2.0, 2.0, 0.01)):
    t.manual_seed(seed)
    L_1, L_2 = t.rand(2, 2)
    P = lambda v: L_1 + v * (L_2 - L_1)
    x, y = zip(P(-2), P(2))
    with fig.batch_update(): 
        fig.data[0].update({"x": x, "y": y}) 
        fig.data[1].update({"x": [L_1[0], L_2[0]], "y": [L_1[1], L_2[1]]}) 
        fig.data[2].update({"x": [P(v)[0]], "y": [P(v)[1]]})

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '530412cd-d1d6-46b9-9bcf-6ea92f5a57f9', 'x': [], 'y': []},
             {'marker': {'size': 12},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '30e8c7b0-5522-4c05-b5d8-088e5daf0b3a',
              'x': [],
              'y': []},
             {'marker': {'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'type': 'scatter',
              'uid': 'a1c55bf4-7ff6-4929-b029-aa0fd55cb9cb',
              'x': [],
              'y': []}],
    'layout': {'height': 500,
               'showlegend': False,
               'template': '...',
               'width': 600,
               'xaxis': {'range': [-1.5, 2.5]},
               'yaxis': {'range': [-1.5, 2.5]}}
})

interactive(children=(IntSlider(value=5, description='seed', max=10), FloatSlider(value=0.0, description='v', …

In [4]:
segments = t.tensor([
    [[1.0, -12.0, 0.0], [1, -6.0, 0.0]], 
    [[0.5, 0.1, 0.0], [0.5, 1.15, 0.0]], 
    [[2, 12.0, 0.0], [2, 21.0, 0.0]]
])
fig = render_lines_with_plotly(t.cat((segments, rays1d), 0))

In [5]:
# @jaxtyped(typeguard.typechecked)
def intersect_ray_1d(ray: Float[Tensor, "2 3"], segment: Float[Tensor, "2 3"]) -> bool:
    '''
    ray: shape (n_points=2, n_dim=3)  # O, D points
    segment: shape (n_points=2, n_dim=3)  # L_1, L_2 points

    Return True if the ray intersects the segment.
    '''
    ray = ray[:, 0:2]
    segment = segment[:, 0:2]
    direction = segment[0]-segment[1]
    stacked = t.stack((ray[1], direction), 1)
    try:
        result = t.linalg.solve(stacked, segment[0]-ray[0])
    except RuntimeError:
        return False
    return (result[0] >= 0 and result[1] <= 1 and result[1] >= 0).item()


tests.test_intersect_ray_1d(intersect_ray_1d)
tests.test_intersect_ray_1d_special_case(intersect_ray_1d)

All tests in `test_intersect_ray_1d` passed!
All tests in `test_intersect_ray_1d_special_case` passed!


D:\Private\Interesting Stuff\Coding\ARENA\ARENA_3.0\chapter0_fundamentals\exercises\part1_ray_tracing\solutions.py:227: UserWarning:

As of jaxtyping version 0.2.24, jaxtyping now prefers the syntax
```
from jaxtyping import jaxtyped
# Use your favourite typechecker: usually one of the two lines below.
from typeguard import typechecked as typechecker
from beartype import beartype as typechecker

@jaxtyped(typechecker=typechecker)
def foo(...):
```
and the old double-decorator syntax
```
@jaxtyped
@typechecker
def foo(...):
```
should no longer be used. (It will continue to work as it did before, but the new approach will produce more readable error messages.)
In particular note that `typechecker` must be passed via keyword argument; the following is not valid:
```
@jaxtyped(typechecker)
def foo(...):
```




In [6]:
x = t.randn(2, 3)
x_repeated = e.repeat(x, 'a b -> a b c', c=4)

assert x_repeated.shape == (2, 3, 4)
for c in range(4):
    t.testing.assert_close(x, x_repeated[:, :, c])

In [7]:
def intersect_rays_1d(rays: Float[Tensor, "nrays 2 3"], segments: Float[Tensor, "nsegments 2 3"]) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if it intersects any segment.
    '''

    nrays = rays.shape[0]
    nsegments = segments.shape[0]

    rays2 = e.repeat(rays, "a b c -> a nsegments b c", nsegments=nsegments)
    segments2 = e.repeat(segments, "a b c -> nrays a b c", nrays=nrays)


    O = rays2[:, :, 0, :2]
    D = rays2[:, :, 1, :2]

    L1 = segments2[:, :, 0, :2]
    L2 = segments2[:, :, 1, :2]


    leftarray = t.stack((D, L1-L2), -1)
    
    rightarray = L1-O

    identity = e.repeat(t.eye(2,2), "a b -> nrays nsegments a b", nrays=nrays, nsegments=nsegments)

    mask = t.linalg.det(leftarray).abs() < 1e-6

    mask = e.repeat(mask, "a b -> a b 2 2")

    leftarray = t.where(mask, identity, leftarray)

    result = t.linalg.solve(leftarray, rightarray)
    
    result = (result[:, :, 0] >= 0) & (result[:, :, 1] <= 1) & (result[:, :, 1] >= 0)

    result = t.where(mask[:, :, 0, 0], False, result)

    result = e.reduce(result.long(), "a b -> a", "sum") > 0.5

    return result


tests.test_intersect_rays_1d(intersect_rays_1d)
tests.test_intersect_rays_1d_special_case(intersect_rays_1d)

All tests in `test_intersect_rays_1d` passed!
All tests in `test_intersect_rays_1d_special_case` passed!


In [10]:
def make_rays_2d(num_pixels_y: int, num_pixels_z: int, y_limit: float, z_limit: float) -> Float[Tensor, "nrays 2 3"]:
    """
    num_pixels_y: The number of pixels in the y dimension
    num_pixels_z: The number of pixels in the z dimension

    y_limit: At x=1, the rays should extend from -y_limit to +y_limit, inclusive of both.
    z_limit: At x=1, the rays should extend from -z_limit to +z_limit, inclusive of both.

    Returns: shape (num_rays=num_pixels_y * num_pixels_z, num_points=2, num_dims=3).
    """


    a = t.zeros(num_pixels_y * num_pixels_z, 1, 3)
    y_vals = e.repeat(t.linspace(-y_limit, y_limit, num_pixels_y), "h -> (i h)", i=num_pixels_z)
    z_vals = e.repeat(t.linspace(-z_limit, z_limit, num_pixels_z), "h -> (h i)", i=num_pixels_y)
    b = t.stack([t.ones(num_pixels_y * num_pixels_z), y_vals, z_vals], -1)
    b = e.repeat(b, "a b -> a 1 b")
    return t.cat([a, b], 1)
    

    


    raise NotImplementedError()


rays_2d = make_rays_2d(10, 10, 0.3, 0.3)
render_lines_with_plotly(rays_2d)

In [14]:
one_triangle = t.tensor([[0, 0, 0], [4, 0.5, 0], [2, 3, 0]])
A, B, C = one_triangle
x, y, z = one_triangle.T

fig = setup_widget_fig_triangle(x, y, z)
display(fig)


@interact(u=(-0.5, 1.5, 0.01), v=(-0.5, 1.5, 0.01))
def update(u=0.0, v=0.0):
    P = A + u * (B - A) + v * (C - A)
    fig.update_traces({"x": [P[0]], "y": [P[1]]}, 2)

FigureWidget({
    'data': [{'marker': {'size': 12},
              'mode': 'markers+text',
              'text': [A, B, C],
              'textfont': {'size': 18},
              'textposition': 'middle left',
              'type': 'scatter',
              'uid': 'bd14e6e5-81c4-4d90-a71d-98fc05dc4a01',
              'x': array([0., 4., 2.], dtype=float32),
              'y': array([0. , 0.5, 3. ], dtype=float32)},
             {'mode': 'lines',
              'type': 'scatter',
              'uid': '394c51e0-88d6-4949-b5f9-b3726807b775',
              'x': [0.0, 4.0, 2.0, 0.0],
              'y': [0.0, 0.5, 3.0, 0.0]},
             {'marker': {'size': 12, 'symbol': 'x'},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '9e200195-7117-4444-a39a-9193ec2a752f',
              'x': [],
              'y': []}],
    'layout': {'height': 600,
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Barycentric c

interactive(children=(FloatSlider(value=0.0, description='u', max=1.5, min=-0.5, step=0.01), FloatSlider(value…

In [56]:
Point = Float[Tensor, "points=3"]


def triangle_ray_intersects(A: Point, B: Point, C: Point, O: Point, D: Point) -> bool:
    """
    A: shape (3,), one vertex of the triangle
    B: shape (3,), second vertex of the triangle
    C: shape (3,), third vertex of the triangle
    O: shape (3,), origin point
    D: shape (3,), direction point

    Return True if the ray and the triangle intersect.
    """
    M_l = t.stack([(-D), B-A, C-A], 1)
    M_r = O-A
    sol = t.linalg.solve(M_l, M_r)
    s, u, v = sol

    return (v.item() >= 0) and (u.item() >= 0) and (u.item()+v.item() <= 1)

tests.test_triangle_ray_intersects(triangle_ray_intersects)

tensor(3.)
tensor(3.)
All tests in `test_triangle_ray_intersects` passed!


In [84]:
def raytrace_triangle(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"], triangle: Float[Tensor, "trianglePoints=3 dims=3"]
) -> Bool[Tensor, "nrays"]:
    """
    For each ray, return True if the triangle intersects that ray.
    """

    nrays = rays.shape[0]
    triangles = e.repeat(triangle, "tp dim -> nrays tp dim", nrays=nrays)

    O = rays[:, 0, :]
    D = rays[:, 1, :]

    A = triangles[:, 0, :]
    B = triangles[:, 1, :]
    C = triangles[:, 2, :]

    M_l = t.stack([(-D), B-A, C-A], 2)
    M_r = O-A

    mask = t.linalg.det(M_l).abs() < 1e-6

    M_l[mask] = t.eye(3)

    sol = t.linalg.solve(M_l, M_r)

    s, u, v = sol.unbind(1)
    
    return (v >= 0) & (u >= 0) & (u+v<= 1)  & (~mask)

    raise NotImplementedError()


A = t.tensor([1, 0.0, -0.5])
B = t.tensor([1, -0.5, 0.0])
C = t.tensor([1, 0.5, 0.5])
num_pixels_y = num_pixels_z = 10
y_limit = z_limit = 0.5

# Plot triangle & rays
test_triangle = t.stack([A, B, C], dim=0)
rays2d = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
triangle_lines = t.stack([A, B, C, A, B, C], dim=0).reshape(-1, 2, 3)
render_lines_with_plotly(rays2d, triangle_lines)

# Calculate and display intersections
intersects = raytrace_triangle(rays2d, test_triangle)
img = intersects.reshape(num_pixels_y, num_pixels_z).int()
imshow(img, origin="lower", width=600, title="Triangle (as intersected by rays)")

In [74]:
def raytrace_triangle_with_bug(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"],
    triangle: Float[Tensor, "trianglePoints=3 dims=3"]
) -> Bool[Tensor, "nrays"]:
    '''
    For each ray, return True if the triangle intersects that ray.
    '''
    NR = rays.shape[0]

    A, B, C = e.repeat(triangle, "pts dims -> pts NR dims", NR=NR)

    O, D = rays.unbind(1)

    mat = t.stack([- D, B - A, C - A], 2)

    dets = t.linalg.det(mat)
    is_singular = dets.abs() < 1e-8
    mat[is_singular] = t.eye(3)

    vec = O - A

    sol = t.linalg.solve(mat, vec)
    s, u, v = sol.unbind(dim=-1)

    return ((u >= 0) & (v >= 0) & (u + v <= 1) & ~is_singular)


intersects = raytrace_triangle_with_bug(rays2d, test_triangle)
img = intersects.reshape(num_pixels_y, num_pixels_z).int()
imshow(img, origin="lower", width=600, title="Triangle (as intersected by rays)")

In [94]:
def raytrace_mesh(
    rays: Float[Tensor, "nrays rayPoints=2 dims=3"], triangles: Float[Tensor, "ntriangles trianglePoints=3 dims=3"]
) -> Float[Tensor, "nrays"]:
    """
    For each ray, return the distance to the closest intersecting triangle, or infinity.
    """
    

    NR = rays.shape[0]

    A, B, C = e.repeat(triangles, "ntr pts dims -> pts nrays ntr dims", nrays=NR)
    O, D = e.repeat(rays, "nrays raypoint dims -> raypoint nrays ntr dims", ntr=A.shape[1])

    mat = t.stack([- D, B - A, C - A], -1)

    dets = t.linalg.det(mat)
    is_singular = dets.abs() < 1e-8
    mat[is_singular] = t.eye(3)

    vec = O - A

    sol = t.linalg.solve(mat, vec)
    s, u, v = sol.unbind(dim=-1)

    s[~((u >= 0) & (v >= 0) & (u + v <= 1) & ~is_singular)] = float("inf")

    return e.reduce(s, "a b -> a", "min")

triangles = t.load(section_dir / "pikachu.pt", weights_only=True)

num_pixels_y = 120
num_pixels_z = 120
y_limit = z_limit = 1

rays = make_rays_2d(num_pixels_y, num_pixels_z, y_limit, z_limit)
rays[:, 0] = t.tensor([-2, 0.0, 0.0])
dists = raytrace_mesh(rays, triangles)
intersects = t.isfinite(dists).view(num_pixels_y, num_pixels_z)
dists_square = dists.view(num_pixels_y, num_pixels_z)
img = t.stack([intersects, dists_square], dim=0)

fig = px.imshow(img, facet_col=0, origin="lower", color_continuous_scale="magma", width=1000)
fig.update_layout(coloraxis_showscale=False)
for i, text in enumerate(["Intersects", "Distance"]):
    fig.layout.annotations[i]["text"] = text
fig.show()